<a href="https://colab.research.google.com/github/shiri9/Federated-Machine-Learning/blob/main/FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q flwr[simulation] torch torchvision matplotlib;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.4/200.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 44.9 MB/s eta 0:00:00


In [4]:
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
#from torchvision.datasets import MNIST
from torch.utils.data import ConcatDataset, DataLoader, Dataset, Subset, random_split

import flwr as fl
from flwr.common import Metrics
from flwr.common.typing import NDArrays, Scalar

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.0.1+cu118 and Flower 1.5.0


In [5]:
SEED = 42
NUM_CLIENTS = 10
BATCH_SIZE = 32
VALID_FRACTION = 0.2 # fraction of the dataset used for each local client
NUM_ROUNDS = 5

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#reading training csv file from google drive
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kdd_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kdd_test.csv')

#creating copies of datasets
df_train_copy = df_train.copy()
df_test_copy = df_test.copy()

#Encoding categories in 0-Normal, 1-DoS, 2-Probe, 3-R2L, 4-U2R
df1_train_labels = df_train_copy['labels']
df1_train_labels_in_numbers = df1_train_labels.replace({ 'normal' : 0, 'neptune' : 1,'land' : 1, 'back': 1, 'teardrop': 1, 'pod': 1, 'smurf' : 1,
                                                     'ipsweep' : 2, 'nmap' : 2, 'portsweep' : 2, 'satan' : 2,
                                                     'phf': 3, 'multihop': 3, 'warezclient': 3,'warezmaster': 3, 'spy': 3, 'ftp_write' : 3,
                                                     'guess_passwd': 3,'imap': 3,
                                                     'buffer_overflow': 4, 'loadmodule': 4,'perl': 4,  'rootkit': 4 })
#replacing the string output clsses by numbers
df_train_copy['labels'] = df1_train_labels_in_numbers

#Encoding categories in 0-Normal, 1-DoS, 2-Probe, 3-R2L, 4-U2R
df1_test_labels = df_test_copy['labels']
df1_test_labels_in_numbers = df1_test_labels.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1,
                                                      'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1,
                                                     'ipsweep' : 2, 'nmap' : 2, 'portsweep' : 2, 'satan' : 2, 'mscan' : 2,'saint' : 2,
                                                     'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3, 'warezclient': 3,
                                                       'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,
                                                       'xsnoop': 3,'httptunnel': 3,
                                                     'buffer_overflow': 4, 'loadmodule': 4,'perl': 4,  'rootkit': 4, 'ps': 4,'xterm': 4 })
#replacing the string output clsses by numbers
df_test_copy['labels'] = df1_test_labels_in_numbers

#Transform categorical features into numbers using LabelEncoder() from train dataset
dft = df_train_copy.apply(LabelEncoder().fit_transform)
df_train_copy = dft

#Transform categorical features into numbers using LabelEncoder() from test dataset
dftt = df_test_copy.apply(LabelEncoder().fit_transform)
df_test_copy = dftt

# Dropping the column "num_outbound_cmds" from train and test datasets
df_train_copy = df_train_copy.drop('num_outbound_cmds', axis=1)
df_test_copy = df_test_copy.drop('num_outbound_cmds', axis=1)

from sklearn.preprocessing import MinMaxScaler

# Select the columns to be normalized (assuming they are numerical features)
numerical_columns = ['duration', 'service', 'flag', 'src_bytes', 'dst_bytes', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'same_srv_rate',
                     'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
                     'dst_host_same_src_port_rate','dst_host_serror_rate', 'dst_host_srv_serror_rate', 'rerror_rate',
                     'srv_rerror_rate','diff_srv_rate', 'srv_diff_host_rate', 'dst_host_srv_diff_host_rate',
                     'dst_host_rerror_rate', 'dst_host_srv_rerror_rate','hot','num_compromised','num_root',]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the selected columns
scaler.fit(df_train_copy[numerical_columns])

# Transform the selected columns with the scaler
df_train_copy[numerical_columns] = scaler.transform(df_train_copy[numerical_columns])

# Select the columns to be normalized (assuming they are numerical features)
numerical_columns = ['duration', 'service', 'flag', 'src_bytes', 'dst_bytes', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'same_srv_rate',
                     'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
                     'dst_host_same_src_port_rate','dst_host_serror_rate', 'dst_host_srv_serror_rate', 'rerror_rate',
                     'srv_rerror_rate','diff_srv_rate', 'srv_diff_host_rate', 'dst_host_srv_diff_host_rate',
                     'dst_host_rerror_rate', 'dst_host_srv_rerror_rate','hot','num_compromised','num_root',]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the selected columns
scaler.fit(df_test_copy[numerical_columns])

# Transform the selected columns with the scaler
df_test_copy[numerical_columns] = scaler.transform(df_test_copy[numerical_columns])

In [26]:
df_test_copy.shape

(22544, 41)

In [15]:
BATCH_SIZE = 32
NUM_CLIENTS = 10


def load_datasets():

    # Assuming "label" is the name of your label column
    df_train_labels = df_train_copy["labels"]
    df_test_labels = df_test_copy["labels"]

    # Convert your DataFrames into PyTorch datasets
    train_dataset = TensorDataset(torch.Tensor(df_train_copy.values), torch.Tensor(df_train_labels.values))
    test_dataset = TensorDataset(torch.Tensor(df_test_copy.values), torch.Tensor(df_test_labels.values))

    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(train_dataset) // NUM_CLIENTS
    # Initialize a list to store the partitions
    data_partitions = []

    # Split the data into partitions
    for i in range(NUM_CLIENTS):
        start_index = i * partition_size
        end_index = (i + 1) * partition_size if i < NUM_CLIENTS - 1 else len(df_train_copy)
        partition = df_train_copy.iloc[start_index:end_index]
        data_partitions.append(partition)

    # Now, data_partitions contains 10 partitions of roughly equal size.

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in data_partitions:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, generator=torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets()

In [19]:
class Net(nn.Module):
    """Basic CNN implementation"""
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding="same")
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding="same")
        self.fc1 = nn.Linear(64 * 7 * 7, 2048)
        self.fc2 = nn.Linear(2048, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 7 * 7 * 64)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

In [20]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = net(inputs)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from flwr.common.typing import NDArray, Scalar
from typing import Dict, Tuple
import numpy as np

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net: nn.Module, trainloader: DataLoader, valloader: DataLoader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self) -> NDArray:
        """Return the current local model parameters as a flat 1D array"""
        return np.concatenate([p.cpu().numpy().flatten() for p in self.net.parameters()])

    def fit(self, parameters: NDArray, config: Dict[str, Scalar]) -> NDArray:
        """Train the model on the local (train) data."""
        # Set the received parameters to the model
        self.set_parameters(parameters)

        # Define the loss and optimizer
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.net.parameters())

        # Training loop
        self.net.train()
        for epoch in range(config['epochs']):
            for inputs, labels in self.trainloader:
                optimizer.zero_grad()
                outputs = self.net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        # Return the updated model parameters
        return self.get_parameters()

    def evaluate(self, parameters: NDArray, config: Dict[str, Scalar]) -> Tuple[float, int, Dict[str, Scalar]]:
        """Evaluate the model using the validation data."""
        # Set the received parameters to the model
        self.set_parameters(parameters)

        # Define the loss and initialize evaluation metrics
        criterion = torch.nn.CrossEntropyLoss()
        total_loss = 0.0
        correct = 0
        total_samples = 0

        # Evaluation loop
        self.net.eval()
        with torch.no_grad():
            for inputs, labels in self.valloader:
                outputs = self.net(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_samples += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Calculate accuracy and return evaluation results
        accuracy = correct / total_samples
        avg_loss = total_loss / len(self.valloader)

        metrics = {"accuracy": accuracy}
        return avg_loss, total_samples, metrics

    def set_parameters(self, parameters: NDArray) -> None:
        """Set the received parameters to the model."""
        offset = 0
        for p in self.net.parameters():
            length = np.prod(p.shape)
            p.data = torch.from_numpy(parameters[offset:offset+length].reshape(p.shape))
            offset += length

In [ ]:
# Create a PyTorch neural network
net = Net().to(DEVICE)

# Train and evaluate the network on each client
for epoch in range(5):
    for i in range(NUM_CLIENTS):
        trainloader = trainloaders[i]
        valloader = valloaders[i]
        train(net, trainloader, epoch)
        loss, accuracy = test(net, valloader)
        print(f"Client {i+1}: validation loss {loss}, accuracy {accuracy}")

# Evaluate the network on the test set
loss, accuracy = test(net, test_dataset)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")